# **1º Create/Update the Server**

The code below will create your server and automatically accept the EULA (dont forget to actually read the EULA !). When you use these code cells, your server will be ready to start!

Update: fixed JsonDecodeError! (It had an error that was becuase paper retired the old api, so we are using a new one!)


**Download/Update the Minecraft Server**

The code below will download Paper, a **very** optimized version of spigot that supports bukkit and spigot!

The server files will be saved in Google Drive (in the Google account you choose or you are currently using).

Also don't forget to check on https://papermc.io/downloads/paper to see if you are updated! :)

In [ ]:
# Available versions:
# - 1.21.4
# - 1.21.3
# - 1.21.2
# - 1.21.1
# - 1.21
# - 1.20.6
# - 1.20.5
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.5
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Newer versions will work too as long as you use the newest version of java

version = '1.9'

# Choose server type. Currently available versions:
# - paper (Most vanilla. It's the same as SPIGOT, bukkit but better and more optimized)
# - forge (For using MODS) [Note, installation takes approximately 10-15 minutes, wait for "COMPLETED" after executing the first step.]
# - fabric (For playing Fabric. It's an alternative to Forge thats "more optimized" and it supports snapshots!)

server_type = 'paper'

from google.colab import drive
import requests
import json

drive.mount("/content/drive", force_remount=True)

# Create the directory which will be used for the server
! mkdir "/content/drive/My Drive/Minecraft-Server"
%cd "/content/drive/My Drive/Minecraft-Server"

paperURL = "https://mcjarfiles.com/api/get-jar/servers/paper/" + version # newer and not retired api :D

forgeURL = "https://mcjarfiles.com/api/get-jar/modded/forge/" + version # now this api is cooler and has the latest version B)

jar_name = {'paper': 'paper.jar', 'fabric': 'fabric-installer.jar', 'forge': 'forge-installer.jar'}
url = {
    'paper': (paperURL),
    'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/1.0.1/fabric-installer-1.0.1.jar',
    'forge': (forgeURL)
    }

print('Downloading to Google Drive...')

r = requests.get(url[server_type])

if r.status_code == 200:
  with open('/content/drive/My Drive/Minecraft-Server/' + jar_name[server_type], 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + '! Most likely you put in a version of the game that doesnt even exist! Try running the code cell again if you think that shouldn\'t have happened :)')

# Running specific install path.
if server_type == 'fabric':
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

if server_type == 'forge':
  %cd "/content/drive/My Drive/Minecraft-Server"
  !java -jar forge-installer.jar --installServer

# Saving config
colabconfig = {"server_type": server_type,
               "server_version": version,}

json.dump(colabconfig, open("colabconfig.json",'w'))

print('Completed Server Install !')

# Go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA! Its gonna be very important if you wanna make a big minecraft server!

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-Server"
!echo "eula=true" >> eula.txt

# **2º** This script will start the server! To stop it click on the same button you used to start it!
[Download PlayIt-GG](https://github.com/playit-cloud/playit-minecraft-plugin/releases/latest/download/playit-minecraft-plugin.jar)

In [1]:
import os
import re
import json
import glob

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, the google colab backend might have to use very old linux packages :("

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-Server"
!ls #list the directory contents (to check if that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exist.
  json.dump(colabconfig, open("colabconfig.json",'w'))


if colabconfig["server_type"] == "forge":
   version = colabconfig["server_version"]
   if colabconfig["server_version"] < "1.17":
    !sudo apt-get install openjdk-15-jre-headless &>/dev/null && echo "Yay! OpenJDK 15 has been successfully installed." || echo "Failed to install OpenJDK 15 (Try again later)"
   else:
    !sudo apt-get install openjdk-21-jre-headless &>/dev/null && echo "Yay! OpenJDK 21 has been successfully installed." || echo "Failed to install OpenJDK 23 (Try again later)."
else:
  !sudo apt-get install openjdk-21-jre-headless &>/dev/null && echo "Yay! OpenJDK 21 has been successfully installed." || echo "Failed to install OpenJDK 23 (Try again later)"

#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "21" :
  print("OpenJDK is working correctly, you're are good to go!")

# Server jar names.
jar_list = {'paper': 'paper.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xms10G -Xmx10G"


!java $memory_allocation $server_flags -jar $jar_name nogui



apt cache successfully updated
Mounted at /content/drive
/content/drive/My Drive/Minecraft-Server
banned-ips.json      config	paper.jar	   version_history.json
banned-players.json  eula.txt	permissions.yml    versions
bukkit.yml	     help.yml	plugins		   whitelist.json
cache		     libraries	server.properties  world
colabconfig.json     logs	spigot.yml	   world_nether
commands.yml	     ops.json	usercache.json	   world_the_end
Yay! OpenJDK 21 has been successfully installed.
OpenJDK is working correctly, you're are good to go!
Starting org.bukkit.craftbukkit.Main
[00:29:12 INFO]: [bootstrap] Running Java 21 (OpenJDK 64-Bit Server VM 21.0.7+6-Ubuntu-0ubuntu122.04; Ubuntu null) on Linux 6.1.123+ (amd64)
[00:29:12 INFO]: [bootstrap] Loading Paper 1.21.4-231-ver/1.21.4@09e9afd (2025-05-19T17:59:36Z) for Minecraft 1.21.4
[00:29:12 INFO]: [PluginInitializerManager] Initializing plugins...
[00:29:17 INFO]: [PluginInitializerManager] Initialized 1 plugin
[00:29:17 INFO]: [PluginInitializerManage

### FAQ - Essential Information

While Google Colab is not made for hosting Minecraft servers. Its is possible to set one up. Here are some things to take away from this:

- **Session Duration**: Google Colab sessions can run for up to 82 hours! but they might turn off the server if the page is inactive for a really long time! To keep your server running (so you players won't be annoyed!) you have to keep your server tab open as much as possible or you can download the colab alive extension!

- **Performance Specs**: Google Colab provides around **12 GB** of RAM, a 2.2 GHz processor with 2 threads, which can outperform many paid hosting services. However, it lacks any support from google, so if anything goes wrong you're on your own!

- **Mod and Plugin Limitations**: While you can install mods and plugins, it’s advisied to **not** overload the server with too many! As this can impact performance! To reduce this lag try to install a plugin like [LightOptimizer](https://www.spigotmc.org/resources/lightoptimizer-%E2%9A%A1-server-performance-booster-fps-helper-%E2%9A%A1.117477/) or a modpack like [1.21 Server Optimizations](https://modrinth.com/collection/YnSBBMnH)

- **Best Use Cases**: If you’re planning to make a small Minecraft server for a few players or a smp (btw call it the colab smp XD) this is the perfect option! But, if you are thinking of making a server with more players, i'd recommend you to either buy a server or self host it on an old computer or cheap computer them you upgrade it to make it possible to host a server!.
